# l2_accuracy_prob_score 
  - module_name: l2_accuracy_prob_score.ipynb
  - read 'Syndigo_GTIN_Mapping_060723.xlsx' and create a base dataframe with column of 'Level 1' and 1SE dfseries 

In [2]:
import numpy as np
import pandas as pd 

pd.options.display.float_format = '{:,.4f}'.format
pd.set_option('display.max_columns', None) 
pd.set_option("display.max_rows", None)
path = 'C:\\users\\iny2819\\kroger\\Data\\'   
import pandas as pd 

pd.options.display.float_format = '{:,.4f}'.format
pd.set_option('display.max_columns', None) 
pd.set_option("display.max_rows", None)
path = 'C:\\users\\iny2819\\kroger\\Data\\'   


In [33]:
def proc_unlabelled_data():
    unlabelled_file   = 'Syndigo_Levels_L1_L2_L3.zip'
    df_unlabelled  = pd.read_csv( path + unlabelled_file, compression="zip") # (1,048,575, 22)
    print(f'df_unlabelled.shape= {df_unlabelled.shape} for unlabelled_file {unlabelled_file}\n')
    print(f'df_unlabelled.columns: \n {df_unlabelled.columns}\n')
    print(f'df_unlabelled.info() {df_unlabelled.info() }\n')
    
    s_unlabelled = df_unlabelled.groupby(['Level1','Level2'])['L2_Scores'].median() 
    df_median = pd.DataFrame (s_unlabelled).rename(columns= {'L2_Scores':'Unlabelled Median' }).reset_index()
    df_cnt = pd.DataFrame(df_unlabelled[['Level1', 'Level2']].value_counts(dropna = True), columns=["Unlabelled Count"]).reset_index()
    df_normd_cnt = pd.DataFrame(df_unlabelled[['Level1', 'Level2']].value_counts(dropna = True,normalize = True), columns=["Unlabelled Nomalized Count"]).reset_index() 

    # sort  files 
    df_median.sort_values(['Level1', 'Level2'], inplace = True)
    print(f'df_median.shape, {df_median.shape}\n')
    print(f'df_median.head(10), {df_median.head(10)}\n') 

    df_cnt.sort_values(['Level1', 'Level2'], inplace = True)
    print(f'df_cnt.shape, {df_cnt.shape}\n')
    print(f'df_cnt.head(10), {df_cnt.head(10)}\n')

    df_normd_cnt.sort_values(['Level1', 'Level2'], inplace = True)
    print(f'df_normd_cnt.shape, {df_normd_cnt.shape}\n')
    print(f'df_normd_cnt.head(10), {df_normd_cnt.head(10)}\n')
 
    # merge files
    df_median_cnt = df_median.merge(df_cnt, on = ['Level1', 'Level2'], how ='left' )  # (157, 4)
    print(f'df_median_cnt.shape, {df_median_cnt.shape}\n')
    print(f'df_median_cnt.head(10), {df_median_cnt.head(10)}') 

    df_median_cnt_normd = df_median_cnt.merge(df_normd_cnt, on = ['Level1', 'Level2'], how ='left' )  # (157, 4)
    print(f'df_median_cnt_normd.shape, {df_median_cnt_normd.shape}\n')
    print(f'df_median_cnt_normd.head(10), {df_median_cnt_normd.head(10)}\n')
    return df_unlabelled, df_median_cnt_normd

In [51]:
def proc_labelled_data():
    labelled_file   = 'syndigo_gtin_level_257K_081523.csv'
    df_labelled  = pd.read_csv( path + labelled_file, dtype='unicode') # (1,048,575, 22)
    print(f'df_labelled.shape= {df_labelled.shape} for labelled_file {labelled_file}\n')
    print(f'df_labelled.columns: \n {df_labelled.columns}\n')
    print(f'df_labelled.info() {df_labelled.info() }\n')
    
    df_cnt = pd.DataFrame(df_labelled[['Level 1', 'Level 2']].value_counts(dropna = True), columns=["labelled Count"]).reset_index()
    df_normd_cnt = pd.DataFrame(df_labelled[['Level 1', 'Level 2']].value_counts(dropna = True,normalize = True), columns=["labelled Nomalized Count"]).reset_index() 

    # sort  files 

    df_cnt.sort_values(['Level 1', 'Level 2'], inplace = True)
    print(f'df_cnt.shape, {df_cnt.shape}\n')
    print(f'df_cnt.head(10), {df_cnt.head(10)}\n')

    df_normd_cnt.sort_values(['Level 1', 'Level 2'], inplace = True)
    print(f'df_normd_cnt.shape, {df_normd_cnt.shape}\n')
    print(f'df_normd_cnt.head(10), {df_normd_cnt.head(10)}\n')
 
    # merge files

    df_cnt_normd = df_cnt.merge(df_normd_cnt, on = ['Level 1', 'Level 2'], how ='left' )  # (157, 4)
    df_cnt_normd.rename(columns = {'Level 1':'Level1' , 'Level 2':'Level2'}, inplace = True)
    print(f'df_cnt_normd.shape, {df_cnt_normd.shape}\n')
    print(f'df_cnt_normd.head(10), {df_cnt_normd.head(10)}\n')
    return df_labelled, df_cnt_normd

In [19]:
def cr_base_df():
    path_file = path + 'Syndigo_GTIN_Mapping_060723.xlsx'
    #syndigo_mapped = pd.read_csv('/dbfs/FileStore/tables/DATA_SCIENCE/Syndigo_GTIN_Mapping_060723.csv')
    syndigo_mapped  = pd.read_excel(path_file)
    #tmp_syn = syndigo_mapped [['Level 1', 'Level 2']].value_counts(dropna = True, ascending = True)  
    s_syn = syndigo_mapped [['Level 1', 'Level 2']].value_counts(dropna = True,sort = False) 
    df_syn_L1_l2 = pd.DataFrame(s_syn,  columns=["Count"]).reset_index() # reset index from series as a column
    df_syn_L1_l2.drop(['Count'], axis=1, inplace = True)
    print(f'df_syn_L1_l2.shape = {df_syn_L1_l2.shape}')
    new_var = ['synd_cnt_train',  'synd_cnt_test', 
           'synd_median_prob_train','synd_median_prob_test',
           'synd_median_accuracy_train','synd_median_accuacy_test',
           'unlabel_median_prob']

    for var in new_var:
         df_syn_L1_l2[var] = np.nan
    print(f'df_syn_L1_l2.head() \n {df_syn_L1_l2.head()}')  
    print(f'df_syn_L1_l2.info() \n {df_syn_L1_l2.info()}')  
    return df_syn_L1_l2 

In [34]:
df_unlabelled, df_merged_unlabelled_median_cnt_normd = proc_unlabelled_data()

df_unlabelled.shape= (5211106, 22) for unlabelled_file Syndigo_Levels_L1_L2_L3.zip

df_unlabelled.columns: 
 Index(['VND_ECOM_DSC', 'Level1', 'L1_Scores', 'Level2', 'L2_Scores', 'GTIN',
       'PMY_DPT_CD', 'PMY_DPT_DSC', 'REC_DPT_CD', 'REC_DPT_DSC', 'DPT_CD',
       'DPT_DSC', 'COM_CD', 'COM_DSC', 'SUBCOM_CD', 'SUBCOM_DSC', 'ITM_ID',
       'L1_Percent', 'L2_Percent', 'L2_Flag', 'Score', 'Level3'],
      dtype='object')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5211106 entries, 0 to 5211105
Data columns (total 22 columns):
 #   Column        Dtype  
---  ------        -----  
 0   VND_ECOM_DSC  object 
 1   Level1        object 
 2   L1_Scores     float64
 3   Level2        object 
 4   L2_Scores     float64
 5   GTIN          float64
 6   PMY_DPT_CD    float64
 7   PMY_DPT_DSC   object 
 8   REC_DPT_CD    float64
 9   REC_DPT_DSC   object 
 10  DPT_CD        float64
 11  DPT_DSC       object 
 12  COM_CD        float64
 13  COM_DSC       object 
 14  SUBCOM_CD     float64
 1

In [52]:
df_labelled, df_merged_labelled_cnt_normd = proc_labelled_data()

df_labelled.shape= (257953, 10) for labelled_file syndigo_gtin_level_257K_081523.csv

df_labelled.columns: 
 Index(['GTIN', 'Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5',
       'Level 6', 'Level 7', 'Level 8', 'Level 9'],
      dtype='object')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257953 entries, 0 to 257952
Data columns (total 10 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   GTIN     257953 non-null  object
 1   Level 1  257953 non-null  object
 2   Level 2  256935 non-null  object
 3   Level 3  247350 non-null  object
 4   Level 4  179389 non-null  object
 5   Level 5  45755 non-null   object
 6   Level 6  5286 non-null    object
 7   Level 7  1542 non-null    object
 8   Level 8  179 non-null     object
 9   Level 9  2 non-null       object
dtypes: object(10)
memory usage: 19.7+ MB
df_labelled.info() None

df_cnt.shape, (279, 3)

df_cnt.head(10),                     Level 1               Level 2  labelled Count
221  

In [53]:

df_labelled_unlabled_medium_cnt_normd = df_merged_labelled_cnt_normd.merge( df_merged_unlabelled_median_cnt_normd, on = ['Level1', 'Level2'], how ='left' )  # (157, 4)
df_labelled_unlabled_medium_cnt_normd_R = df_merged_labelled_cnt_normd.merge( df_merged_unlabelled_median_cnt_normd, on = ['Level1', 'Level2'], how ='right' )


In [55]:
df_labelled_unlabled_medium_cnt_normd_R.head(10)

,Level1,Level2,labelled Count,labelled Nomalized Count,Unlabelled Median,Unlabelled Count,Unlabelled Nomalized Count
0,Agricultural Equipment,Tires,2,0.0000,0.5588,25,0.0000
1,Apparel,Clothing,863,0.0034,0.7098,561297,0.1077
2,Apparel,Footwear,110,0.0004,0.8650,599,0.0001
3,Apparel,Personal Accessories,237,0.0009,0.4995,9230,0.0018
4,Apparel,Safety Apparel,73,0.0003,0.4328,85,0.0000
5,Appliances,Parts & Accessories,24,0.0001,0.5965,7,0.0000
6,Appliances,Small Appliances,651,0.0025,0.9409,21406,0.0041
7,Arts & Crafts,Craft Kits,106,0.0004,0.2542,8870,0.0017
8,Arts & Crafts,Drawing Tools,294,0.0011,0.2622,66271,0.0127
9,Arts & Crafts,Needlework & Toy Making,40,0.0002,0.2913,82,0.0000


In [54]:
df_labelled_unlabled_medium_cnt_normd.head(10)

,Level1,Level2,labelled Count,labelled Nomalized Count,Unlabelled Median,Unlabelled Count,Unlabelled Nomalized Count
0,Agricultural Equipment,Tires,2,0.0000,0.5588,25.0000,0.0000
1,Agricultural Equipment,Vehicle Parts,1,0.0000,NaN,NaN,NaN
2,Apparel,Clothing,863,0.0034,0.7098,"561,297.0000",0.1077
3,Apparel,Footwear,110,0.0004,0.8650,599.0000,0.0001
4,Apparel,Personal Accessories,237,0.0009,0.4995,"9,230.0000",0.0018
5,Apparel,Safety Apparel,73,0.0003,0.4328,85.0000,0.0000
6,Appliances,Cooktops,2,0.0000,NaN,NaN,NaN
7,Appliances,Microwaves,5,0.0000,NaN,NaN,NaN
8,Appliances,Parts & Accessories,24,0.0001,0.5965,7.0000,0.0000
9,Appliances,Ranges,5,0.0000,NaN,NaN,NaN


In [58]:
path = 'C:\\users\\iny2819\\kroger\\Data\\'
df_labelled_unlabled_medium_cnt_normd.to_excel(path + "accuracy_cnt_normd_median.xlsx",sheet_name = 'L', index=None)
df_labelled_unlabled_medium_cnt_normd_R.to_excel(path + "accuracy_cnt_normd_median_R.xlsx",sheet_name = 'R',index=None)

In [59]:
df_labelled_unlabled_medium_cnt_normd.isnull().sum()

Level1                          0
Level2                          0
labelled Count                  0
labelled Nomalized Count        0
Unlabelled Median             122
Unlabelled Count              122
Unlabelled Nomalized Count    122
dtype: int64